In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from src.etl import export_dict_to_feather

In [2]:
df = pd.read_csv('data/Data raw.csv', delimiter=';')


In [3]:
lista_columnas = [" FECHA ", " INGRESOS_MATRICULAS ", " INGRESOS_RENOVACIONES ", " INGRESOS_ESAL ", " MATRICULA_COMERCIANTE ", " RENOVADOS_ESAL ", " RENOVADOS_COMERCIANTE "]

In [4]:
df = df[lista_columnas]

In [5]:
# Renombrar columnas para mejor legibilidad
df = df.rename(columns={
    ' FECHA ': 'Fecha',
    ' INGRESOS_MATRICULAS ': 'Ingresos Matrículas',
    ' INGRESOS_RENOVACIONES ': 'Ingresos Renovaciones',
    ' INGRESOS_ESAL ': 'Ingresos ESAL',
    ' MATRICULA_COMERCIANTE ': 'Matrícula Comerciante',
    ' RENOVADOS_ESAL ': 'Renovados ESAL',
    ' RENOVADOS_COMERCIANTE ': 'Renovados Comerciante'
})

In [6]:
# Función para limpiar y convertir valores monetarios a numéricos
def clean_currency_column(series):
    """
    Convierte una serie con formato de moneda a valores numéricos
    Maneja formatos como: $540,778,126, $ 738,00, etc.
    """
    if series.dtype == 'object':
        # Remover símbolos de moneda, espacios, y comas
        cleaned = series.astype(str)
        cleaned = cleaned.str.replace('$', '', regex=False)
        cleaned = cleaned.str.replace(',', '', regex=False)
        cleaned = cleaned.str.strip()
        
        # Reemplazar valores vacíos o 'nan' con NaN
        cleaned = cleaned.replace(['', 'nan', 'None'], pd.NA)
        
        # Convertir a numérico
        try:
            return pd.to_numeric(cleaned, errors='coerce')
        except:
            return series
    else:
        return series

# Aplicar limpieza a todas las columnas excepto Fecha

for col in df.columns:
    if col != 'Fecha':
        original_dtype = df[col].dtype
        df[col] = clean_currency_column(df[col])
       

In [7]:
# Convertir la columna de fecha a datetime
print("Convirtiendo columna de fecha...")
# Option 1: Convert to Period for proper monthly representation
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%m/%d/%Y', errors='coerce').dt.to_period('M')


Convirtiendo columna de fecha...


In [8]:
# Crear diccionario donde cada elemento contiene fecha y una columna
# La llave es el nombre de la columna

diccionario_datos = {}

# Obtener las columnas excepto 'FECHA'
columnas_datos = [col for col in df.columns if col != 'Fecha']

# Crear un elemento del diccionario para cada columna
for columna in columnas_datos:
    diccionario_datos[columna] = df[['Fecha', columna]].copy()



In [9]:
# Eliminar filas con valores nulos en cada DataFrame
for columna in diccionario_datos.keys():
    # Eliminar filas donde cualquier columna tenga valores nulos
    diccionario_datos[columna] = diccionario_datos[columna].dropna()



In [10]:

# EJECUTAR LA FUNCIÓN CON EL DICCIONARIO
manifest_resultado = export_dict_to_feather(
    data_dict=diccionario_datos,
    base_path="data/feather",
    manifest_path="data/feather_manifest.json",
    compression="lz4",
    category_threshold=0.5
)

